# Probit Example for Moment Sensitivity

This example is a replication of the probit model in [Honoré, Jørgensen & de Paula (2020)](https://papers.ssrn.com/abstract=3518640), Section 3.1.

Through this example, we will show you how to set up the `params`, how to use the`optimize` functions, and how to calculate the local `moment_sensitivity` measurements for your own model. 

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

from estimagic import minimize
from estimagic.config import TEST_DIR
from estimagic.sensitivity.moment_sensitivity import moment_sensitivity

ModuleNotFoundError: No module named 'estimagic.sensitivity.moment_sensitivity'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from estimagic.visualization.sensitivity_plot import moment_sensitivity_plot

sns.set(style="whitegrid")

## Input Data

First, we should create the index for `params`. Here, we only have one set of params, `beta`. 

In [ ]:
params_index = [["beta"], ["intercept", "x1", "x2"]]
params_index = pd.MultiIndex.from_product(params_index, names=["type", "name"])
print(params_index)

Now read the data.

In [ ]:
data = pd.read_csv(TEST_DIR / "sensitivity" / "sensitivity_probit_example_data.csv")
y = data[["y"]]
y.head()

In [ ]:
x = data[["intercept", "x1", "x2"]]
x.head()

## Calculate the Moments

In [ ]:
def calc_moments_value(params, x, y, estimate_y=True):
    """This is the func2 in sensitivity module.

    Args:
        params (pd.DataFrame): see :ref:`params`
        x (pd.DataFrame)
        y (pd.DataFrame)
        estimate_y (boolean): use estimated y_star

    Return:
        mom_value (pd.DataFrame): sample value of moments
    """

    if estimate_y == True:
        y_estimated = x.to_numpy() @ (params["value"].to_numpy())
    else:
        y_estimated = y.copy(deep=True)

    x_np = x.T.to_numpy()

    residual = y.T.to_numpy() - stats.norm.cdf(y_estimated)
    mom_value = []

    # loop through all x

    length = len(x_np)

    for i in range(length):
        for j in range(i, length):
            moment = residual * x_np[i] * x_np[j]
            mom_value.append(moment)

    mom_value = np.stack(mom_value, axis=1)[0]
    mom_value = pd.DataFrame(data=mom_value)

    return mom_value

In [ ]:
def calc_moments_expectation(params, x, y, estimate_y=True):
    """This is the func1 in sensitivity module.

    Args:
        params (pd.DataFrame): see :ref:`params`
        x (pd.DataFrame)
        y (pd.DataFrame)
        estimate_y (boolean): use estimated y_star

    Return:
        moments (np.array): expectation of moments
    """

    mom_value = calc_moments_value(params, x, y, estimate_y)

    moments = mom_value.mean(axis=1)

    # do not use pd.DataFrame since jacobian: extrapolation=False
    # moments = pd.DataFrame(data=moments,
    #                        columns=["moments"])

    return moments

## Criterion Function & Estimation

This is the criterion function:

In [ ]:
def criterion_func(params, x, y, weight_matrix):
    """
    Args:
        params (pd.DataFrame)
        x (pd.DataFrame)
        y (pd.DataFrame)
        weight_matrix (np.array)

    Return:
        criterion (float)
    """

    moments = calc_moments_expectation(params, x, y, estimate_y=True).to_numpy()

    criterion = moments.T @ weight_matrix @ moments
    criterion = criterion.item()

    return criterion

Give the initial arguments:

In [ ]:
I_matrix = np.identity(6)

params_init = pd.DataFrame(
    data=np.full((3, 1), 0.5), index=params_index, columns=["value"]
)
params_init

Now, we can estimate the `params` using identity matrix as the weight matrix:

In [ ]:
estimation = minimize(
    criterion=criterion_func,
    params=params_init,
    criterion_kwargs={"x": x, "y": y, "weight_matrix": I_matrix},
    algorithm="scipy_lbfgsb",
)

In [ ]:
params_estimated = estimation["solution_params"]
params_estimated

The true `beta`'s for this data are $1/\sqrt{3}\approx 0.57735$, so that looks ok for the small sample size.

## Sensitivity Measurements

Now, we can call the sensitivity function without specifying a weight matrix, in other words, using the optimal weight matrix.

In [ ]:
sensitivity = moment_sensitivity(
    moment_func=calc_moments_expectation,
    moment_contributions_func=calc_moments_value,
    params=params_estimated,
    kwargs={"x": x, "y": y},
)

In [ ]:
sensitivity

## Visualization

In [ ]:
moment_sensitivity_plot(sensitivity)